# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Phan Trí Nguyên
- MSSV: 20127578
- Lớp: 20CLC05

# Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import random

# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [2]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1].to_numpy()    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1].to_numpy()     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần
label = train.columns

# Cài đặt hàm

In [3]:
# Cài đặt các hàm cần thiết ở đây
x_train_2d = np.array(X_train)
y_train_2d = np.array(y_train)

x_test_2d = np.array(X_test)
y_test_2d = np.array(y_test)

class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self


    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1) 

# Yêu cầu 1a: Sử dụng toàn bộ 10 đặc trưng đề bài cung cấp (2 điểm) 

In [4]:
# Phần code cho yêu cầu 1a
lr = OLSLinearRegression().fit(X_train, y_train)
print(list(lr.get_params()), "\n")

[0.015101362735318279, 0.09021998065775627, 0.04292181752549435, 0.13928911689488216, -0.5673328270884068, -0.0001007651148748953, 0.7407134377587112, 0.19093579767396474, 24.505973591149445, 2.393516607832779] 



In [5]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
y_test_preds = lr.predict(X_test)

#cách 1: dùng thư viện có sẵn
def RMSE(y, y_preds):
    return mean_squared_error(y, y_preds, squared = False)
#có square để làm key, nếu là False thì phép tính đã có căn bậc hai

#cách 2: dùng thư viện có sẵn
def RMSE_2(y, y_preds):
     # nếu là True thì phép tính chưa có căn bậc hai
    mse = mean_squared_error(y_test, y_test_preds, squared = True)
    return (np.sqrt(mse))

# cách 3: viết hàm tính toán
def RMSE_3(y, y_preds):
    return np.sqrt(np.mean((y.ravel() - y_preds.ravel())**2))

print('Dùng thư viện có sẵn với squared là False:\n RMSE = ', RMSE(y_test, y_test_preds), '\n')
print('Dùng thư viện có sẵn với squared là True:\n RMSE 2 = ', RMSE_2(y_test, y_test_preds), '\n')
print('Hàm xử lí tính toán:\n RMSE 3 = ', RMSE_3(y_test, y_test_preds), "\n")

Dùng thư viện có sẵn với squared là False:
 RMSE =  7.064046430584705 

Dùng thư viện có sẵn với squared là True:
 RMSE 2 =  7.064046430584705 

Hàm xử lí tính toán:
 RMSE 3 =  7.064046430584705 



Công thức hồi quy

$$\text{Life expectancy} = (0.0151 * Adult Mortality) + (0.0902 * BMI) + (0.0429 *  Polio) + (0.1392 * Diphtheria) + (-0.5673 * HIV/AIDS) + (-0.0001 * GDP) + (0.7407 * Thinness age 10-19) + (0.1909 * Thinness age 5-9) + (24.5059 * Income composition of resources) + (2.3935 * Schooling)$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng, tìm mô hình cho kết quả tốt nhất (2 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [6]:
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
def CrossValidation(x_test, y_test, fold):
    rmse = 0
    n = x_test.shape[0]
    delta = n/fold
   
    for i in range (fold):
        start_sample = int(i*delta)
        end_sample = int((i+1)*delta)
        
        x_value = x_test[start_sample:end_sample]
        y_value = y_test[start_sample:end_sample]
        
        lr.fit(x_value, y_value)
        
        y_train_pred = lr.predict(x_value)

        x_test_feature = np.array([*x_test[:start_sample], *x_test[end_sample:]])
        y_test_feature = np.array([*y_test[:start_sample], *y_test[end_sample:]])
        
        rmse += RMSE(y_value, y_train_pred)
        
    return rmse/fold

def shuffle_matrix():
    return random.shuffle(mlist)

def append_data(myList):
    x_train_temp = []
    y_train_temp = []
    for i in range(X_train.shape[0]):
        x_train_temp.append(x_train_2d[myList[i]])
        y_train_temp.append(y_train_2d[myList[i]])

    x_train_temp = np.array(x_train_temp)
    y_train_temp = np.array(y_train_temp) 
    return x_train_temp, y_train_temp

def calculate_RMSE():
    for i in range(number_of_feature):
        x_train_feature = x[:, i].copy().reshape((-1, 1))
        y_train_feature = y.copy()
        rmse[i] = CrossValidation(x_train_feature, y_train_feature, 5)

'''-----------------------------------'''
# khai báo các biến cần thiết để tính toán
x = []
y = []
mlist = list(range(0, X_train.shape[0]))
rmse = np.zeros(10)
number_of_feature = 10

'''------Excute program------'''

# xáo trộn dữ liệu trong danh sách 1 lần duy nhất 
shuffle_matrix()
x, y = append_data(mlist)

# hàm thực hiện việc tính toán RMSE của 10 đặc trưng
calculate_RMSE()

index_best_feature = np.where(rmse == np.amin(rmse))[0][0]
    
'''------Print table------'''    
output = pd.DataFrame(y_test[0:10])
output['Mô hình với 1 đặc trưng'] = label[0:10]
output['RMSE'] = rmse[0:10]
print(output, "\n")

'''------Print result------'''
print("Index of the best feature: ", index_best_feature, "\n")

   Life expectancy          Mô hình với 1 đặc trưng       RMSE
0             65.0                  Adult Mortality  46.012537
1             59.9                              BMI  27.926360
2             77.8                            Polio  18.047051
3             77.5                       Diphtheria  16.004723
4             75.4                         HIV/AIDS  66.737829
5             51.7                              GDP  60.083419
6             76.2               Thinness age 10-19  51.579988
7             74.6                 Thinness age 5-9  51.410890
8             82.7  Income composition of resources  13.289349
9             81.4                        Schooling  11.767871 

Index of the best feature:  9 



In [7]:
# Huấn luyện lại mô hình best_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
# câu 1b ý 4
X_train_best_feature = x[:, index_best_feature].copy().reshape((-1,1))
lr.fit(X_train_best_feature, y)
print(list(lr.get_params()), "\n")

[5.5573993976919205] 



In [8]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_feature_model
best_feature_model = lr.predict(X_test.to_numpy()[:, index_best_feature].reshape(-1, 1))
print("RMSE = ", RMSE(y_test.copy(), best_feature_model), "\n")

RMSE =  10.26095039165537 



Công thức hồi quy

$$\text{Life expectancy} =5.5573994 * Schooling$$

# Yêu cầu 1c: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

In [9]:
# tính toán để chọn ra Hệ số tương quan, những số liệu đặc trưng nào có Hệ số tương quan gần nhau\
# thì có chỉ số thống kê đo lường mức độ mạnh yếu của mối quan hệ giữa các biến số
corr = 0
for i in range (10):
    corr = np.corrcoef(x_train_2d[:,i], y_train_2d)[0, 1]
    print(corr)
    
print("\n")

-0.6852900668760471
0.6106869049325893
0.36570463313391055
0.40679787281341856
-0.5865777200247838
0.4867608561696112
-0.503652334646405
-0.5002309457544403
0.779403572108283
0.754863721105356




In [10]:
# 2 đặc trưng cho mô hình: Thinness age 5-9, Income composition of resources
           #đặc trưng thứ: 7 và 8
'''-----------------------------------'''
# khai báo các biến cần thiết để tính toán    
x_train_1 = []
rmse1 = np.zeros(10)

def calculate_RMSE_2feature():
    x_train_feature_age_5_9 = x[:, 7].copy().reshape((-1, 1))
    x_train_feature_Income = x[:, 8].copy().reshape((-1, 1))
    
    x_train_feature1 = np.column_stack((x_train_feature_age_5_9, x_train_feature_Income)) # hàm nối 2 cột với nhau
    y_train_feature = y.copy()
    rmse1 = CrossValidation(x_train_feature1, y_train_feature, 5)
    
    return x_train_feature1, rmse1

x_train_1, rmse1 = calculate_RMSE_2feature()

# In ra các kết quả
print("RMSE 1 = ", rmse1, "\n")

RMSE 1 =  11.476463331521298 



In [11]:
# 3 đặc trưng cho mô hình: Polio, Diphtheria, GDP
           #đặc trưng thứ: 2, 3 và 5
'''-----------------------------------'''
rmse2 = np.zeros(10)
x_train_2 = []

def calculate_RMSE_3feature():
    x_train_feature_Polio = x[:, 2].copy().reshape((-1, 1))
    x_train_feature_Diphtheria = x[:, 3].copy().reshape((-1, 1))
    x_train_feature_GDP = x[:, 5].copy().reshape((-1, 1))

    x_train_feature2 = np.column_stack((x_train_feature_Polio, x_train_feature_Diphtheria, x_train_feature_GDP))
    y_train_feature = y.copy()

    rmse2 = CrossValidation(x_train_feature2, y_train_feature, 5)
    return x_train_feature2, rmse2

x_train_2, rmse2 = calculate_RMSE_3feature()

# In ra các kết quả
print("RMSE 2 = ", rmse2, "\n")

RMSE 2 =  14.605660520221926 



In [12]:
# 5 đặc trưng cho mô hình: BMI, Polio, Diphtheria, GDP, Schooling
           #đặc trưng thứ: 1, 2, 3, 5 và 9
'''-----------------------------------'''
rmse3 = np.zeros(10)
x_train_3 = []

def calculate_RMSE_5feature():
    X_train_feature_BMI= x[:, 1].copy().reshape((-1, 1))
    X_train_feature_Polio = x[:, 2].copy().reshape((-1, 1))
    X_train_feature_Diphtheria = x[:, 3].copy().reshape((-1, 1))
    X_train_feature_GDP = x[:, 5].copy().reshape((-1, 1))
    X_train_feature_Schooling = x[:, 9].copy().reshape((-1, 1))

    X_train_feature3 = np.column_stack((X_train_feature_BMI, 
                                        X_train_feature_Polio, 
                                        X_train_feature_Diphtheria,
                                        X_train_feature_GDP, 
                                        X_train_feature_Schooling))
    y_train_feature = y.copy()

    rmse3 = CrossValidation(X_train_feature3, y_train_feature, 5)

    return X_train_feature3, rmse3

x_train_3, rmse3 = calculate_RMSE_5feature()

# In ra các kết quả
print("RMSE 3 = ", rmse3, "\n")

RMSE 3 =  9.577930755622955 



In [13]:
'''------Print array------'''
def find_min_rmse():
    min = rmse1
    if rmse2 < min: min = rmse2
    if rmse3 < min: min = rmse3
    return min


min = find_min_rmse()
if min == rmse1:
    X_train_best_feature = x_train_1

if min == rmse2:
    X_train_best_feature = x_train_2

if min == rmse3:
    X_train_best_feature = x_train_3

print(min, "\n")


lr.fit(X_train_best_feature, y)
print(list(lr.get_params()), "\n")

9.577930755622955 

[0.03518197210685069, 0.07866293712201275, 0.186681017407329, -0.00022255201233355718, 3.7819384704454277] 



In [14]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
X_train_feature_BMI = X_test.to_numpy()[:, 1].copy().reshape((-1, 1))
X_train_feature_Polio = X_test.to_numpy()[:, 2].copy().reshape((-1, 1))
X_train_feature_Diphtheria = X_test.to_numpy()[:, 3].copy().reshape((-1, 1))
X_train_feature_GDP = X_test.to_numpy()[:, 5].copy().reshape((-1, 1))
X_train_feature_Schooling = X_test.to_numpy()[:, 9].copy().reshape((-1, 1))

connect_X_train_best = np.column_stack((X_train_feature_BMI, 
                                        X_train_feature_Polio, 
                                       X_train_feature_Diphtheria,
                                       X_train_feature_GDP,
                                       X_train_feature_Schooling))
y_best_pred = lr.predict(connect_X_train_best)
print("RMSE = ", RMSE(y_test.copy(), y_best_pred), "\n")

RMSE =  8.559353765137367 



Công thức hồi quy

$$\text{Life expectancy} = (BMI *0.0351) + (Polio * 0.0786) + (Diphtheria * 0.1866) + (GDP * (-0.0002)) + (Schooling * 3.7819)$$